In [1]:
using Pkg
Pkg.activate("libs/")
# Pkg.instantiate()
# Pkg.add("MLUtils")
using CSV
using JLD2
using CUDA
using Glob
using Dates
using Zygote
using DICOM
using Images
using MLUtils
using Setfield
using ImageView
using ImageDraw
using Statistics
using DataFrames
using StaticArrays
using MLDataPattern
using ChainRulesCore
using Distributions: Normal
using FastAI, FastVision, Flux, Metalhead
import CairoMakie; CairoMakie.activate!(type="png")

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/molloi-lab/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Check how many threads we have

In [2]:
Threads.nthreads()

64

List all CUDA devices

In [3]:
CUDA.allowscalar(false)
CUDA.devices()

CUDA.DeviceIterator() for 4 devices:
0. NVIDIA GeForce RTX 4090
1. NVIDIA GeForce RTX 4090
2. NVIDIA GeForce RTX 4090
3. NVIDIA GeForce RTX 4090

# Notes

1. In this training, images' pixel values are zoomed into range = [0, 1]. This should not be applied to final plan since different images have different min and max.

2. In this training, only `pixel values` are being used. In future, we can feed more features to the NN to get a better information like `L_mass` or `R_mass`.

In [4]:
patch_size = 256
patch_size_half = round(Int, patch_size/2);

# Helper functions

In [5]:
"""
    This function zoom all pixel values into [0, 1].
"""
function zoom_pxiel_values(img)
    a, b = minimum(img), maximum(img)
    img = (img .- a) / (b - a)
    return img
end

zoom_pxiel_values

In [6]:
"""
    This function takes in a img of various size, 
    returns patches with size = patch_size * patch_size.
"""
function patch_image(img, lbl)
    s = size(img)
    x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
    y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
    num_patches = x*y
    img_patches = Array{Float32, 4}(undef, patch_size, patch_size, 1, num_patches)
    lbl_patches = Array{Float32, 4}(undef, patch_size, patch_size, 1, num_patches)
    ct = 0
    for i = 1 : x-1
        x_start = 1+(i-1)*patch_size_half
        x_end = x_start+patch_size-1
        for j = 1 : y-1
            y_start = 1+(j-1)*patch_size_half
            y_end = y_start+patch_size-1
            # save patch
            ct += 1
            img_patches[:, :, 1, ct] = img[x_start:x_end, y_start:y_end]
            lbl_patches[:, :, 1, ct] = lbl[x_start:x_end, y_start:y_end]
        end
        # right col
        y_start, y_end = s[2]-patch_size+1, s[2]
        # save patch
        ct += 1
        img_patches[:, :, 1, ct] = img[x_start:x_end, y_start:y_end]
        lbl_patches[:, :, 1, ct] = lbl[x_start:x_end, y_start:y_end]
    end
    # last row
    x_start, x_end = s[1]-patch_size+1, s[1]
    for j = 1 : y-1
        y_start = 1+(j-1)*patch_size_half
        y_end = y_start+patch_size-1
        # save patch
        ct += 1
        img_patches[:, :, 1, ct] = img[x_start:x_end, y_start:y_end]
        lbl_patches[:, :, 1, ct] = lbl[x_start:x_end, y_start:y_end]
    end
    # right col
    y_start, y_end = s[2]-patch_size+1, s[2]
    # save patch
    ct += 1
    img_patches[:, :, 1, ct] = img[x_start:x_end, y_start:y_end]
    lbl_patches[:, :, 1, ct] = lbl[x_start:x_end, y_start:y_end]
    # return
    return num_patches, img_patches, lbl_patches
end

patch_image

In [7]:
"""
    This function fixs the path to the images and labels.
"""
function fix_path!(data_set)
    num_data = size(data_set)[1]
    Threads.@threads for i = 1 : num_data
        for j = 1 : 2
            for k = 1 : 4
                # modify img path
                splited = split(deepcopy(data_set[i][j][k]), "\\")
                if size(splited)[1] > 1
                    new_path = joinpath("../collected_dataset_for_ML", joinpath(splited[4:end]))
                    data_set[i][j][k] = new_path
                end
            end
        end
    end
end

fix_path!

In [8]:
"""
    This function check how many number of images and labels there will be after patching.
"""
function get_num_of_imgs(data_set)
    num_data = size(data_set)[1]
    cts = Array{Int}(undef, num_data*4)
    Threads.@threads for i = 1 : num_data
        @views t = train_set[i]
        for j = 1 : 4
            # read dicom images
            s = size(dcm_parse(t[1][j])[(0x7fe0, 0x0010)])
            x = ceil(Int, s[1]/patch_size) + floor(Int, (s[1]-patch_size_half)/patch_size)
            y = ceil(Int, s[2]/patch_size) + floor(Int, (s[2]-patch_size_half)/patch_size)
            # save 
            cts[(i-1)*4+j] = x*y
        end
    end
    return cts
end

get_num_of_imgs

# 1. Prepare

In [9]:
@load "clean_set_step2_for_ubuntu.jld2" train_set valid_set

2-element Vector{Symbol}:
 :train_set
 :valid_set

In [10]:
data_dir = "../collected_dataset_for_ML";

Check if dataset is found

In [11]:
isdir(data_dir)

true

## 1.1 Load train set & valid set
container format: patch_size * patch_size * 1 * num_imgs

In [12]:
# get num of total patches(train)
ct_patches_train = get_num_of_imgs(train_set)
num_patches_train = sum(ct_patches_train)

703276

In [13]:
# runtime: 30s
num_train_data = size(train_set)[1]
train_container_images = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_train)
train_container_masks = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_train)
Threads.@threads for i = 1 : num_train_data
    start_idx = sum(ct_patches_train[1:i-1])+1
    for j = 1 : 4 # 4 images each patient
        # read dicom images
        img = Float16.(dcm_parse(train_set[i][1][j])[(0x7fe0, 0x0010)])
        # read png images
        lbl = Float16.(Images.load(train_set[i][2][j]))
        # process image
        num_patches, img_patches, lbl_patches = patch_image(img, lbl)
        # save 
        end_idx = start_idx+num_patches-1
        train_container_images[:, :, 1, start_idx : end_idx] = img_patches
        train_container_masks[:, :, 1, start_idx : end_idx] = lbl_patches
        start_idx = end_idx
    end
end

In [14]:
# get num of total patches(valid)
ct_patches_valid = get_num_of_imgs(valid_set)
num_patches_valid = sum(ct_patches_valid)

113564

In [15]:
# runtime: 5s
num_valid_data = size(valid_set)[1]
valid_container_images = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_valid)
valid_container_masks = Array{Float16, 4}(undef, patch_size, patch_size, 1, num_patches_valid)
Threads.@threads for i = 1 : num_valid_data
    start_idx = sum(ct_patches_valid[1:i-1])+1
    for j = 1 : 4 # 4 images each patient
        # read dicom images
        img = Float16.(dcm_parse(valid_set[i][1][j])[(0x7fe0, 0x0010)])
        # read png images
        lbl = Float16.(Images.load(valid_set[i][2][j]))
        # process image
        num_patches, img_patches, lbl_patches = patch_image(img, lbl)
        # save 
        end_idx = start_idx+num_patches-1
        valid_container_images[:, :, 1, start_idx : end_idx] = img_patches
        valid_container_masks[:, :, 1, start_idx : end_idx] = lbl_patches
        start_idx = end_idx
    end
end

In [16]:
GC.gc(true)

## 1.2 Create dataloaders

In [17]:
b_s = 10
batch_size = b_s*4
# batch_size = 20
train_loader = MLUtils.DataLoader((data=train_container_images, label=train_container_masks), batchsize=batch_size)
test_loader = MLUtils.DataLoader((data=valid_container_images, label=valid_container_masks), batchsize=b_s);

In [18]:
for (x, y) in train_loader
    global x_sample = x[:, :, :, 1:b_s]
    global y_sample = y[:, :, :, 1:b_s]
    println(size(x))
    println(typeof(x))
    println(size(y))
    println(typeof(y))
    break
end

(256, 256, 1, 40)
Array

{Float16, 4}
(256, 256, 1, 40)
Array{Float16, 4}


## 1.3 Create Model

In [19]:
function _random_normal(shape...)
    return Float16.(rand(Normal(0.0,0.02),shape...))
end

# _conv = (stride, in, out) -> Conv((3, 3), in=>out, stride=stride, pad=SamePad();init=_random_normal)
# _tran = (stride, in, out) -> ConvTranspose((2, 2), in=>out, stride=stride, pad=SamePad();init=_random_normal)
_conv = (stride, in, out) -> Conv((3, 3), in=>out, stride=stride, pad=SamePad())
_tran = (stride, in, out) -> ConvTranspose((2, 2), in=>out, stride=stride, pad=SamePad())

conv1 = (in, out) -> Chain(_conv(1, in, out), BatchNorm(out, leakyrelu))
conv2 = (in, out) -> Chain(_conv(2, in, out), BatchNorm(out, leakyrelu))
conv3 = (in, out) -> Chain(_conv(1, in, out), x -> softmax(x; dims = 3))
# conv3 = (in, out) -> Chain(_conv(1, in, out), sigmoid)
tran2 = (in, out) -> Chain(_tran(2, in, out), BatchNorm(out, leakyrelu))



function unet2D(in_chs, lbl_chs)
    # Contracting layers
    l1 = Chain(conv1(in_chs, 64), conv1(64, 64))
    l2 = Chain(l1, MaxPool((2,2), stride=2), conv1(64, 128), conv1(128, 128))
    l3 = Chain(l2, MaxPool((2,2), stride=2), conv1(128, 256), conv1(256, 256))
    l4 = Chain(l3, MaxPool((2,2), stride=2), conv1(256, 512), conv1(512, 512))
    l5 = Chain(l4, MaxPool((2,2), stride=2), conv1(512, 1024), conv1(1024, 1024), tran2(1024, 512))

    # Expanding layers
    l6 = Chain(Parallel(FastVision.Models.catchannels,l5,l4), 
                conv1(512+512, 512),
                conv1(512, 512),
                tran2(512, 256))
    l7 = Chain(Parallel(FastVision.Models.catchannels,l6,l3), 
                conv1(256+256, 256),
                conv1(256, 256),
                tran2(256, 128))
    l8 = Chain(Parallel(FastVision.Models.catchannels,l7,l2), 
                conv1(128+128, 128),
                conv1(128, 128),
                tran2(128, 64))
    l9 = Chain(Parallel(FastVision.Models.catchannels,l8,l1), 
                conv1(64+64, 64),
                conv1(64, 64),
                conv3(64, lbl_chs))
end

unet2D (generic function with 1 method)

## 1.4 Create Loss

In [20]:
function dice_loss(ŷ, y; ϵ=1f-5)
    # ŷ, y = Float32.(ŷ), Float32.(y)
    loss_dice = 0f0
    for chan_idx = 1 : 2
        @inbounds loss_dice += 
        1f0 - (muladd(2f0, sum(ŷ[:,:,chan_idx,:] .* y[:,:,chan_idx,:]), ϵ) / (sum(ŷ[:,:,chan_idx,:] .^ 2) + sum(y[:,:,chan_idx,:] .^ 2) + ϵ))
    end
    return loss_dice/2f0
end
lossfn = dice_loss
# lossfn = Flux.Losses.dice_coeff_loss

dice_loss (generic function with 1 method)

## 1.5 Loop

In [30]:
function data_parallel(device_id, x, y, model)
	y_cated = cat(Float16(1) .- y, y, dims = 3)
	device!(device_id)
	x = x |> gpu
	y_cated = y_cated |> gpu
	ls, gs = Flux.withgradient(Flux.params(model)) do 
		lossfn(model(x), y_cated)
	end
	CUDA.unsafe_free!(x)
	CUDA.unsafe_free!(y_cated)
	return ls, gs
end

# function update_params!(id_from, id_to, ps_from, ps_to)
# 	Threads.@threads for i = 1 : 90
# 		device!(id_from)
# 		p = ps_from[i] |> cpu
# 		device!(id_to)
# 		# ps_to[i] .= nothing
# 		ps_to[i] .= p |> gpu
# 	end
# end

function gradient_to_array(device_id, gs)
    device!(device_id)
    gs_array_gpu = [gs[p] for p in gs.params]
	gs_array_cpu = gs_array_gpu |> cpu
	Threads.@threads for i = 1 : 90
		CUDA.unsafe_free!(gs_array_gpu[i])
	end
	for p in gs.params
		CUDA.unsafe_free!(gs.grads[p])
		# CUDA.unsafe_free!(p)
	end

    device!(0)
    return gs_array_cpu |> gpu
end

function train_1_epoch!(epoch_idx, model_gpu0, model_gpu1, model_gpu2, model_gpu3, train_dl, optimizer)
	
	# Epoch start
	losses = Float32[]
	step_ct = 0
	for (x, y) in train_dl
		gs, gs_gpu0, gs_gpu1, gs_gpu2, gs_gpu3 = nothing, nothing, nothing, nothing, nothing
		ls0, ls1, ls2, ls3 = nothing, nothing, nothing, nothing
		@sync begin
			# Step start
			@async begin
				x_gpu0, y_gpu0 = x[:, :, :, 1:10], y[:, :, :, 1:10]
				ls0, gs = data_parallel(0, x_gpu0, y_gpu0, model_gpu0)
				gs_gpu0 = [gs[p] for p in gs.params]
			end
			@async begin
				x_gpu1, y_gpu1 = x[:, :, :, 11:20], y[:, :, :, 11:20]
				ls1, gs_gpu1_temp = data_parallel(1, x_gpu1, y_gpu1, model_gpu1)
				gs_gpu1 = gradient_to_array(1, gs_gpu1_temp)
			end
			@async begin
				x_gpu2, y_gpu2 = x[:, :, :, 21:30], y[:, :, :, 21:30]
				ls2, gs_gpu2_temp = data_parallel(2, x_gpu2, y_gpu2, model_gpu2)
				gs_gpu2 = gradient_to_array(2, gs_gpu2_temp)
			end
			@async begin
				x_gpu3, y_gpu3 = x[:, :, :, 31:end], y[:, :, :, 31:end]
				ls3, gs_gpu3_temp = data_parallel(3, x_gpu3, y_gpu3, model_gpu3)
				gs_gpu3 = gradient_to_array(3, gs_gpu3_temp)
			end
		end
		push!(losses, (ls0+ls1+ls2+ls3)/4)
		ls0, ls1, ls2, ls3 = nothing, nothing, nothing, nothing
		
		@info "step $step_ct\tloss = $(mean(losses))"
		losses = Float32[]

		device!(0)
		gs_gpu0 = gs_gpu0 .+ gs_gpu1 .+ gs_gpu2 .+ gs_gpu3

		Threads.@threads for i = 1 : 90
			gs.grads[Flux.params(model_gpu0)[i]] .= gs_gpu0[i]
		end
		
	  	Flux.update!(optimizer, Flux.params(model_gpu0), gs)
		model_cpu = model_gpu0 |> cpu

		# sync new params to GPUs
		@sync begin
			@async begin
				device!(0)
				for p in gs.params
					CUDA.unsafe_free!(gs.grads[p])
					# CUDA.unsafe_free!(p)
				end
				Threads.@threads for i = 1 : 90
					CUDA.unsafe_free!(gs_gpu0[i])
				end
			end
			@async begin
				device!(1)
				Threads.@threads for i = 1 : 90
					CUDA.unsafe_free!(gs_gpu1[i])
				end
				model_gpu1 = nothing
				model_gpu1 = model_cpu |> gpu
				# model_ps_gpu1 = Flux.params(model_gpu1)
				# update_params!(0, 1, model_ps_gpu0, model_ps_gpu1)
			end
			@async begin
				device!(2)
				Threads.@threads for i = 1 : 90
					CUDA.unsafe_free!(gs_gpu2[i])
				end
				model_gpu2 = nothing
				model_gpu2 = model_cpu |> gpu
				# model_ps_gpu2 = Flux.params(model_gpu2)
				# update_params!(0, 2, model_ps_gpu0, model_ps_gpu2)
			end
			@async begin
				device!(3)
				Threads.@threads for i = 1 : 90
					CUDA.unsafe_free!(gs_gpu3[i])
				end
				model_gpu3 = nothing
				model_gpu3 = model_cpu |> gpu
				# model_ps_gpu3 = Flux.params(model_gpu3)
				# update_params!(0, 3, model_ps_gpu0, model_ps_gpu3)
			end
		end
		gs, gs_gpu0, gs_gpu1, gs_gpu2, gs_gpu3 = nothing, nothing, nothing, nothing, nothing
		
	  	# Step finished
		step_ct += 1
		if step_ct % 1000 == 0 
			@info "step $step_ct\tloss = $(mean(losses))"
			losses = Float32[]
		end
		flush(stdout)
		break
	end
	# Epoch finished
	# if epoch_idx % 5 == 0
	# 	@info "loss = $(mean(losses))"
	# end
	# return model_ps_gpu0, model_ps_gpu1, model_ps_gpu2, model_ps_gpu3
end

train_1_epoch! (generic function with 1 method)

In [22]:
# function report_loss(model, train_dl, valid_dl)
# 	# train set
# 	losses = []
# 	for (x, y) in train_dl
#         ignore_derivatives() do
#             y_cated = cat(Float16(1) .- y, y, dims = 3)
#             x = x |> gpu
#             pred_y = model(x) |> cpu
#             loss = lossfn(pred_y, y_cated)
#             push!(losses, loss)
#         end
# 	end
# 	println("train Loss = $(mean(losses))")
	
# 	# valid set
# 	losses = []
# 	for (x, y) in valid_dl
#         ignore_derivatives() do
#             y_cated = cat(Float16(1) .- y, y, dims = 3)
#             x = x |> gpu
#             pred_y = model(x) |> cpu
#             loss = lossfn(pred_y, y_cated)
#             push!(losses, loss)
#         end
# 	end
# 	println("Valid Loss = $(mean(losses))")
#     flush(stdout)
# end

# 2. Train

In [23]:
# lossfn = dice_loss
model = unet2D(1, 2);

In [24]:
# model_test = model |> gpu
# y_sample_cated = cat(Float16(1) .- y_sample, y_sample, dims = 3)
# x_sample = x_sample |> gpu
# y_sample_cated = y_sample_cated |> gpu
# # gs = Flux.gradient(Flux.params(model_test)) do 
# ls, gs = Flux.withgradient(Flux.params(model_test)) do 
#     lossfn(model_test(x_sample), y_sample_cated)
# end

In [25]:
# device!(3)
# model_test3 = model |> gpu
# y_sample_cated3 = cat(Float16(1) .- y_sample, y_sample, dims = 3)
# x_sample3 = x_sample |> gpu
# y_sample_cated3 = y_sample_cated3 |> gpu
# # gs3 = Flux.gradient(Flux.params(model_test3)) do 
# ls3, gs3 = Flux.withgradient(Flux.params(model_test3)) do 
#     lossfn(model_test3(x_sample3), y_sample_cated3)
# end

In [26]:
model_test = nothing
x_sample = nothing
y_sample = nothing
device!(0)
CUDA.reclaim()
device!(1)
CUDA.reclaim()
device!(2)
CUDA.reclaim()
device!(3)
CUDA.reclaim()
GC.gc(true)
device!(0)

CuDevice(0): NVIDIA GeForce RTX 4090

In [27]:
device!(0)
model_0 = model |> gpu
# model_ps_0 = Flux.params(model_0)
device!(1)
model_1 = model |> gpu
# model_ps_1 = Flux.params(model_1)
device!(2)
model_2 = model |> gpu
# model_ps_2 = Flux.params(model_2)
device!(3)
model_3 = model |> gpu
# model_ps_3 = Flux.params(model_3);
""

""

In [28]:
# @sync begin
#     # Step start
#     @async begin
#         device!(0)
#         global model_0 = model |> gpu
#         global model_ps_0 = Flux.params(model_0)
#     end
#     @async begin
#         device!(1)
#         global model_1 = model |> gpu
#         global model_ps_1 = Flux.params(model_1)
#     end
#     @async begin
#         device!(2)
#         global model_2 = model |> gpu
#         global model_ps_2 = Flux.params(model_2)
#     end
#     @async begin
#         device!(3)
#         global model_3 = model |> gpu
#         global model_ps_3 = Flux.params(model_3)
#     end
# end
# modelss = [model_0, model_1, model_2, model_3]
# model_pss = [model_ps_0, model_ps_1, model_ps_2, model_ps_3];

In [29]:
optimizer = AdaGrad(0.01)
for epoch_idx = 1:15
    @info epoch_idx
    # model_pss = train_1_epoch(epoch_idx, modelss, model_pss, train_loader, optimizer)
    train_1_epoch!(epoch_idx, model_0, model_1, model_2, model_3, train_loader, optimizer)
    flush(stdout)
end

┌ Info: 1
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/4_train.ipynb:3


┌ Info: step 0	loss = 1.0
└ @ Main /home/molloi-lab/Desktop/Project BAC/BAC project/4_train.ipynb:74


ErrorException: Invalid input to `update!`.
* For the implicit style, this needs `update(::AbstractOptimiser, ::Params, ::Grads)`
* For the explicit style, `update(state, model, grad)` needs `state = Flux.setup(opt, model)`.
